In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import torch.nn as nn
import torch
import sklearn.preprocessing as preprocessing
import os

from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader


import warnings
warnings.filterwarnings(action='ignore')
from copy import deepcopy

from Prescriptor import Prescriptor
from Evolution.crossover import UniformCrossover, WeightedSumCrossover
from Evolution.mutation import MultiplyNormalMutation, MultiplyUniformMutation, AddNormalMutation, AddUniformMutation, ChainMutation, FlipSignMutation
from Evolution.selection import RouletteSelection, TournamentSelection, ParetoLexsortSelection
from Evolution import Evolution

In [2]:
# train.py

import os
import torch
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

from data_loader import load_data_1m
from feature_calculations import (
    resample_data, calculate_MA_data, calculate_ema_bollinger_bands, calculate_rsi,
    calculate_macd, calculate_stochastic_oscillator, calculate_adx, calculate_atr,
    calculate_obv, calculate_williams_r, base_feature_fn, cyclic_encode_fn, log_transform
)
from strategies import BB_fitness_fn
from dataset import make_dataset, replace_nan_with_zero
from train_functions_bi_cul import inference, fitness_fn, generation_valid, generation_test, calculate_fitness

from Prescriptor import Prescriptor
from Evolution.crossover import UniformCrossover, WeightedSumCrossover
from Evolution.mutation import MultiplyNormalMutation, MultiplyUniformMutation, AddNormalMutation, AddUniformMutation, ChainMutation, FlipSignMutation
from Evolution.selection import RouletteSelection, TournamentSelection, ParetoLexsortSelection
from Evolution import Evolution




In [3]:
state_dict_path = '/root/daily/bit_4/generation/generation_13.pt'
if os.path.exists(state_dict_path):
    state_dict = torch.load(state_dict_path)
    start_gen_idx = state_dict['generation'] + 1
    best_profit = state_dict['best_profit']
    best_chromosomes = state_dict['best_chromosomes']
    # prescriptor.load_state_dict(state_dict['prescriptor_state_dict'],strict=True)

In [4]:
best_profit

tensor([[ 4.4455,  2.8229,  0.5556, 28.2320, 13.2333],
        [ 3.5243,  4.5946,  0.6061, 13.2833,  7.0554],
        [ 4.9981,  3.9588,  0.5600, 23.8934,  6.2675],
        [ 1.0030,  2.1835,  0.5144, 37.5944,  6.9246],
        [ 2.4961,  3.1338,  0.5269, 27.4748,  7.1995],
        [ 7.0274,  4.6758,  0.5686, 28.8293, 14.4837],
        [ 9.8918,  4.6710,  0.4828, 38.9116, 10.9394],
        [ 5.4200,  5.2849,  0.6667, 23.9151,  9.0904],
        [ 3.2488,  2.5634,  0.5652, 39.0162,  6.9345],
        [ 6.7967,  3.7302,  0.4342, 37.3415, 20.6907],
        [ 3.6240,  2.5610,  0.5054, 38.1069, 11.6277],
        [ 9.4688,  6.3219,  0.6111, 37.8078, 45.2324],
        [ 8.3365,  4.3218,  0.5200, 36.4893,  9.6903],
        [ 7.1229,  3.7023,  0.4211, 34.2280, 11.4782],
        [ 5.2992,  6.1765,  0.5088, 13.2449, 11.6851],
        [ 4.0012,  3.5328,  0.4800, 37.2042,  9.3934],
        [ 1.2465,  2.6854,  0.4852, 21.7424,  7.9292],
        [ 3.6511,  2.7779,  0.4767, 39.2351, 10.3216],
        [ 

In [10]:
optimal_solution = best_profit[(best_profit[:, 4] > 0.8) & (best_profit[:, 3] <= 40)]
optimal_index = torch.where((best_profit[:, 4] > 0.8) & (best_profit[:, 3] <= 40))[0]

In [11]:
optimal_index

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [19]:
fitness_index

array([11,  7,  5,  1,  6,  9,  2, 12,  0, 13,  4, 10,  8,  3])

In [12]:
fitness = calculate_fitness(deepcopy(optimal_solution.numpy()))
fitness_index = np.argsort(-fitness)

In [13]:
index = 0

In [14]:
optimal_solution[fitness_index][:, 0][index]

tensor(9.4688)

In [15]:
optimal_solution[fitness_index][:, 1][index]

tensor(6.3219)

In [16]:
optimal_solution[fitness_index][:, 2][index]

tensor(0.6111)

In [17]:
optimal_solution[fitness_index][:, 3][index]

tensor(37.8078)

In [18]:
optimal_solution[fitness_index][:, 4][index]

tensor(45.2324)